# Further Tuning of the ML Model for the InsuranceCharges Data

Since we saw that the model does not perform differently including sex, we will remove it from our data. In `ML-DBmodel.ipynb` we saw that the LinearRegression (i.e. Lasso with $\alpha$ = 0) was the best performing model.

In this notebook we will try to improve the $R^2$ value of the LinearRegression model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
#For connecting with SQL database
import pymssql
from config import database
from config import username
from config import password
from config import server

In [ ]:
#Create connectiong to databse
conn = pymssql.connect(server, username, password, database)
cursor = conn.cursor()

In [ ]:
#Run the query to gather the table
table = 'dbo.InsuranceCharges'

querycosts = '''Select I.ChargeID, I.ChargeValue, I.AgeID, A.AgeLabel, I.ChildrenID, C.ChildrenLabel, I.RegionID, 
R.RegionLabel, I.SexID, S.SexLabel, I.SmokerID, Sm.SmokerLabel, I.BMI from InsuranceCharges I
inner join Age A on I.AgeID = A.AgeID
inner join Children C on  I.ChildrenID = C.ChildrenID
inner join Region R on  I.RegionID = R.RegionID
inner join Sex S on  I.SexID = S.SexID
inner join Smoker Sm on  I.SmokerID = Sm.SmokerID
'''
#Load the query to a pandas dataframe
df_costs = pd.read_sql(querycosts, conn)
df_costs

In [ ]:
df = df_costs[['AgeLabel', 'BMI', 'ChildrenLabel', 'RegionLabel', 'ChargeValue', 'SmokerLabel']]
df

In [ ]:
df['AgeLabel'] = df['AgeLabel'].astype('int64')
df['ChildrenLabel'] = df['ChildrenLabel'].astype('int64')

In [ ]:
df_dummies = pd.get_dummies(df, columns = ['RegionLabel', 'SmokerLabel'], drop_first = True)

In [ ]:
df_dummies

Now we isolate the target variable (`ChargeValue`) and assign it as `y`, which is a culmination of individual medical costs incurred by people with attributes `X` (i.e. `AgeLabel`, `BMI`, `ChildrenLabel`, `RegionaLabel`, `SmokerLabel`).

In [ ]:
X = df_dummies.drop(columns = 'ChargeValue').copy()
y = df_dummies[['ChargeValue']].copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 9)

In [ ]:
model = make_pipeline(PolynomialFeatures(degree=2),
                      LinearRegression())

In [ ]:
param_grid = {'polynomialfeatures__degree': np.arange(8)}

grid = GridSearchCV(model, param_grid, cv=7)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

The best params are
* polynomialfeatures_degree: 3

In [ ]:
best_model = grid.best_estimator_

In [ ]:
best_model.score(X_test, y_test)

With approximately 10 percentage point increase the new model has an $R^2$ score of 87%

This was mostly achieved through the use of polynomialfeatures as we saw the alphas made little-to-no difference near $\alpha$ = 0

We can use the model to create predictions for potential insurance customers, but first we export the model using joblib

### Joblib Machine Learning Model Export

In [ ]:
from joblib import dump, load

In [ ]:
dump(best_model, 'LinearRegressionTunedJoblib.model')

In [ ]:
load_model = load('LinearRegressionTunedJoblib.model')

In [ ]:
X_test.head()

In [ ]:
#Create a new entry to predict the costs

prediction1 = {'AgeLabel': 22,
              'BMI': 22.3,
              'ChildrenLabel': 0,
              'RegionLabel_northwest': 0,
              'RegionLabel_southeast': 1,
              'RegionLabel_southwest': 0,
              'SmokerLabel_True': 0}


In [ ]:
test_prediction_df = pd.DataFrame(prediction1, index=[1])
test_prediction_df

In [ ]:
load_model.predict(test_prediction_df)

A 22 year-old living in the southeast with a BMI of 22.3 and no children will incur 3473.15 dollars of healthcare costs